In [1]:
import numpy as np
import re
import pandas as pd
import pyarrow.parquet as pq
from collections import defaultdict
from math import log

import gensim

In [2]:
file_path_video = 'videos.parquet'
df_v = pd.read_parquet(file_path_video)

In [3]:
file_path_player = 'player_starts_train.parquet'
hack = pd.read_parquet(file_path_player)

In [4]:
data = pd.merge(hack, df_v, on="item_id", how="inner")
data = data.dropna()

In [ ]:
# emotions = pd.read_csv("emotions.csv")
# emotions.rename(columns = {"date": "emotion_date"}, inplace=True)
# emotions.rename(columns = {"user_id": "emotion_user_id"}, inplace=True)
# data = pd.merge(data, emotions, on="item_id", how="left")

In [5]:
# Фильтруем строки, где watch_time не равен 0 и не равен 1
filtered_data = data[(data['watch_time'] != 0) & (data['watch_time'] != 1)]

# Условие 1: Если duration / 1000 >= 3000 и watch_time <= 0.25 * duration
condition1 = (data['duration'] / 1000 >= 3000) & (data['watch_time'] <= 0.25 * data['duration'])

# Условие 2: Если (duration / 1000 < 3000 и watch_time <= 30 и duration / 10000 != watch_time)
condition2 = ((data['duration'] / 1000 < 3000) & (data['watch_time'] <= 30 & (data['duration']/1000 != data['watch_time'])))

# Условие 3: Если тип эмоции негативный
# condition3 = (data['type'] == 'neg_emotions')

# Фильтруем строки, которые не соответствуют ни одному из условий
# data = data[~(condition1 | condition2 | condition3)]
data = filtered_data[~(condition1 | condition2)]

/var/folders/zz/zyxvpxvq6csfxvn_n003wgq800z3xt/T/ipykernel_73349/1942235562.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = filtered_data[~(condition1 | condition2)]


In [6]:
user_id_to_process = 'user_19364826'  # Замените 'your_user_id_here' на конкретный user_id
user_data = data[data['user_id'] == user_id_to_process]

user_data = user_data.sort_values(by='date', ascending=False)

top_10_items = user_data.head(10)['item_id'].unique()

In [13]:
df_v['text_category'] = df_v['video_description'].fillna('') + ' ' + df_v['video_title']
text = df_v.groupby('category_title')['text_category'].apply(lambda x: x.tolist()).reset_index()

In [14]:
with open('stop.txt', 'r') as file:
    stop_word = file.read().split()

In [15]:
def clear_text(text_list):
    list_filtered_words = []
    for text in text_list:
        words = re.split(r'\s+|\n+', text)
        filtered_words = []
        for word in words:
            if len(word) >= 2 and word not in stop_word:
                word = str(word).lower()
                first_char = word[0]
                last_char = word[-1]
                if not last_char.isalnum():
                    word = word[:-1]
                if not first_char.isalnum():
                    word = word[1:]
                filtered_words.append(word)
        ' '.join(filtered_words)
        list_filtered_words.append(filtered_words)
    return list_filtered_words

In [ ]:
text['text_category'] = text['text_category'].apply(clear_text)

In [ ]:
df = pd.DataFrame(columns=['category_title', 'tag', 'koef'])
for index, theme in enumerate(text['text_category']):
    tf = defaultdict(int) 
    idf = defaultdict(int) 
    koef = {}
    count_file = len(theme)
    count_word = 0
    for document in theme:
        count_word += len(document)
        for word in set(document):
            tf[word] += document.count(word)
            idf[word] += 1
    for word in tf:
        koef[word] = (tf[word] / count_word) * (count_file / (idf[word] + 1))
    sorted_koef = dict(sorted(koef.items(), key=lambda item: item[1], reverse=True))
    is_end = 0
    for key, value in sorted_koef.items():
        if is_end < 100 and re.search(r'[a-zа-я0-9]', key) is not None:
            df = pd.concat([df, pd.DataFrame({'category_title': [text['category_title'][index]], 'tag': [key], 'koef': [value]})])
            is_end += 1

In [17]:
path_tags = "tags.csv"
# df.to_csv(path_tags)
df = pd.read_csv(path_tags)

In [ ]:
list_tags_each_video = []
for index_1, text in enumerate(df_v['text_category']):
    for word in text:
        list_tag = []
        for index_2, title in enumerate(df['category_title']):
            if df_v['category_title'][index_1] == df['category_title'][index_2]:
                for tag in df['tag']:
                    if word == tag: 
                        list_tag.append(tag)
    list_tags_each_video.append(list_tag)

In [39]:
def text_vec(text):
    res = np.zeros(50)
    cnt = 0
    for word in text:
        try:
            res += model.wv[word]
            cnt += 1
        except:
            pass
    if cnt > 0:
        return res / (cnt * np.linalg.norm(res / cnt))
    else:
        return res

In [20]:
tags = []
tags_from_csv = df.groupby('category_title')['tag'].apply(lambda x: x.tolist()).reset_index()
for item in tags_from_csv['tag']:
    tags.append(item)

In [ ]:
model = gensim.models.FastText(list_tags_each_video, min_count=1, negative=5, size=50, window=5, workers=4)
model.wv.vectors_vocab = None
model.wv.vectors = model.wv.vectors.astype(np.float16)
model.wv.vectors_ngrams = model.wv.vectors_ngrams.astype(np.float16)

In [ ]:
texts_mean_vect = [text_vec(items) for items in list_tags_each_video]
np_texts_mean_vect = np.vstack(texts_mean_vect)

In [ ]:
vect_query = text_vec(query.split()).reshape(-1,1) / np.linalg.norm(text_vec(query.split()))
relevance = np.matmul(np_texts_mean_vect, vect_query)